In [ ]:
from Bio import PDB as bio
import numpy as np
import pymol_helper as ph
import math
from scipy.spatial.distance import pdist, squareform 
from collections import defaultdict

In [ ]:
protein = bio.MMCIFParser().get_structure('7kkk', '7kkk.cif')
model = protein[0]
chainA = model['A']
residues = list(chainA.get_residues())

/opt/miniconda3/envs/pymol/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 52224.
  warnings.warn(
/opt/miniconda3/envs/pymol/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 52504.
  warnings.warn(
/opt/miniconda3/envs/pymol/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 52784.
  warnings.warn(


#We want to get all the SG atom coordinates subsetting to those within a distance threshold but I also want to know the adjacent carbon atoms.  

In [3]:
model = protein.get_models()

In [4]:
atoms = protein.get_atoms()
type(atoms)
atoms_list= list(atoms)

In [6]:
# the following code looks up the cys and methionine residues and gets the gamma sulfurs from them 
# then pairs it with the beta carbon to keep as pairs to calculate dihedral angles downstream
res_lookup = defaultdict(dict)
for atom in atoms_list:
    res = atom.get_parent()  # parent is the Residue
    resname = res.get_resname()
    resid = res.get_id()[1]  # tuple like (' ', 45, ' '), so take the residue number
    atom_name = atom.get_name()
    
    res_lookup[(resname, resid)][atom_name] = atom
    
sulfur_pairs = []
for (resname, resid), atomdict in res_lookup.items():
    if resname == "CYS" and "SG" in atomdict and "CB" in atomdict:
        sulfur_pairs.append({
            "resname": resname,
            "resid": resid,
            "s_atom": atomdict["SG"],
            "c_atom": atomdict["CB"]
        })

for p in sulfur_pairs:
    print(f"{p['resname']} {p['resid']}: {p['s_atom'].get_name()} ↔ {p['c_atom'].get_name()}")

CYS 131: SG ↔ CB
CYS 136: SG ↔ CB
CYS 166: SG ↔ CB
CYS 291: SG ↔ CB
CYS 301: SG ↔ CB
CYS 336: SG ↔ CB
CYS 361: SG ↔ CB
CYS 379: SG ↔ CB
CYS 391: SG ↔ CB
CYS 432: SG ↔ CB
CYS 480: SG ↔ CB
CYS 488: SG ↔ CB
CYS 525: SG ↔ CB
CYS 538: SG ↔ CB
CYS 590: SG ↔ CB
CYS 617: SG ↔ CB
CYS 649: SG ↔ CB
CYS 662: SG ↔ CB
CYS 671: SG ↔ CB
CYS 738: SG ↔ CB
CYS 743: SG ↔ CB
CYS 749: SG ↔ CB
CYS 760: SG ↔ CB
CYS 1032: SG ↔ CB
CYS 1043: SG ↔ CB
CYS 1082: SG ↔ CB
CYS 1126: SG ↔ CB
CYS 22: SG ↔ CB
CYS 96: SG ↔ CB


In [7]:
# the following code block only gets sulfur atoms and not beta carbons 
Sulfur_atoms = []
for atom in atoms_list: 
    if atom.element == "S":
        sulfur_atom = atom
        sulfur_coord = atom.get_coord()
        print(sulfur_atom, sulfur_coord)
        Sulfur_atoms.append([sulfur_atom, sulfur_coord])        


<Atom SG> [256.555 225.139 253.756]
<Atom SG> [266.938 235.768 250.015]
<Atom SG> [257.004 223.205 254.359]
<Atom SG> [241.023 226.562 215.046]
<Atom SG> [241.373 226.289 213.075]
<Atom SG> [206.637 244.75  248.838]
<Atom SG> [206.105 245.514 247.031]
<Atom SG> [211.587 229.933 246.42 ]
<Atom SG> [210.244 244.107 237.745]
<Atom SG> [209.961 231.16  246.397]
<Atom SG> [174.658 214.42  261.106]
<Atom SG> [174.506 213.424 262.869]
<Atom SG> [209.313 245.606 238.826]
<Atom SG> [223.739 245.742 220.17 ]
<Atom SG> [222.009 245.29  219.218]
<Atom SG> [230.8   248.117 202.768]
<Atom SG> [231.605 246.485 203.67 ]
<Atom SG> [231.522 234.091 188.644]
<Atom SG> [231.003 236.02  189.102]
<Atom SD> [228.049 236.385 186.762]
<Atom SD> [224.287 207.737 196.403]
<Atom SG> [219.197 198.829 214.198]
<Atom SD> [228.944 196.211 216.65 ]
<Atom SG> [221.593 199.71  221.613]
<Atom SG> [221.988 200.964 223.208]
<Atom SG> [217.478 199.435 215.078]
<Atom SD> [230.96  190.952 186.245]
<Atom SD> [229.088 201.401 1

In [8]:
print(Sulfur_atoms[0])
atom1 = Sulfur_atoms[0]
print(atom1[1])
print(type(atom1[1]))

[<Atom SG>, array([256.555, 225.139, 253.756], dtype=float32)]
[256.555 225.139 253.756]
<class 'numpy.ndarray'>


In [9]:
print(sulfur_pairs)

[{'resname': 'CYS', 'resid': 131, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 136, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 166, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 291, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 301, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 336, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 361, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 379, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 391, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 432, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 480, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 488, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname': 'CYS', 'resid': 525, 's_atom': <Atom SG>, 'c_atom': <Atom CB>}, {'resname':

In [10]:
def distance(coord1, coord2):
    dist = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])+ (coord1[2] - coord2[2]))
    return abs(dist)

In [11]:
coordsList = []
for a in sulfur_pairs:
    cys_id = a['resid']
    s = a['s_atom']
    s_coord = s.get_coord()
    coordsList.append(s_coord)

coords2 = np.vstack([np.asarray(a, dtype=float) for a in coordsList])


In [13]:
threshold = 2.1


# coords = np.vstack([np.asarray(a[1], dtype=float) for a in Sulfur_atoms])

dist_condensed = pdist(coords2)

dist_matrix = squareform(dist_condensed)

idx, jdx = np.triu_indices_from(dist_matrix, k = 1)
mask = dist_matrix[idx, jdx] <= threshold

results = []

for i, j, d in zip(idx[mask], jdx[mask], dist_matrix[idx[mask], jdx[mask]]):
    results.append({
        "atom1_index": i,
        "atom1_type": Sulfur_atoms[i][0],
        "atom2_index": j,
        "atom2_type": Sulfur_atoms[j][0],
        "distance": d
    })

for r in results:
    print(f"{r['atom1_type']}({r['atom1_index']}) - {r['atom2_type']}({r['atom2_index']}) : {r['distance']:.2f} Å")


<Atom SG>(0) - <Atom SG>(2) : 2.08 Å
<Atom SG>(3) - <Atom SG>(4) : 2.02 Å
<Atom SG>(5) - <Atom SG>(6) : 2.03 Å
<Atom SG>(7) - <Atom SG>(9) : 2.04 Å
<Atom SG>(8) - <Atom SG>(12) : 2.07 Å
<Atom SG>(10) - <Atom SG>(11) : 2.03 Å
<Atom SG>(13) - <Atom SG>(14) : 2.03 Å
<Atom SG>(15) - <Atom SG>(16) : 2.03 Å
<Atom SG>(17) - <Atom SG>(18) : 2.04 Å
<Atom SD>(19) - <Atom SD>(22) : 2.02 Å
<Atom SD>(20) - <Atom SG>(21) : 2.07 Å
<Atom SG>(23) - <Atom SG>(24) : 2.02 Å
<Atom SG>(25) - <Atom SD>(26) : 2.02 Å
